In [1]:
%cd ..

/home/ltorres/leo/tesis/cloud-classification/code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from torchvision.io import read_image
import glob
import os
import math


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from random import sample
import torch
import torch.nn as nn

In [4]:
from src.dataset import GCD
from src import config
from src import engine_gnn as engine
from src import utils
from src import sweeps

from src.models.graph_nets import GraphConvGNN, GATConvGNN

In [5]:
from sklearn.metrics import accuracy_score

In [6]:
import wandb

In [7]:
wandb.login()

wandb: Currently logged in as: leonardtd (use `wandb login --relogin` to force relogin)


True

In [8]:
device = config.DEVICE

### Sweep config

In [9]:
sweep_config = {
    'method': 'random'
    }

In [10]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [11]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd', 'nadam']
        },
    'criterion': {
        'values': ['cross_entropy', 'loge']
        },
    'hidden_dim': {
        'values': [128, 256, 512, 1024]
        },
    'num_hidden': {
        'values': [2,3,4]
        },
    'num_heads': {
          'values': [1,2,4]
        },
    'similarity_threshold': {
          'values': [0.5, 0.6, 0.7, 0.75, 0.8, 0.9]
        },
    }

sweep_config['parameters'] = parameters_dict

In [12]:
parameters_dict.update({
    'epochs': {
        'value': config.EPOCHS,
        },
    'device': {
        'value': device,
        },
    })

In [13]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 1e-6,
        'max': 6e-5
      },
    'batch_size': {
        # integers between 16 and 32
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(16),
        'max': math.log(32),
      }
    })

In [14]:
sweep_config

{'method': 'random',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'optimizer': {'values': ['adam', 'sgd', 'nadam']},
  'criterion': {'values': ['cross_entropy', 'loge']},
  'hidden_dim': {'values': [128, 256, 512, 1024]},
  'num_hidden': {'values': [2, 3, 4]},
  'num_heads': {'values': [1, 2, 4]},
  'similarity_threshold': {'values': [0.5, 0.6, 0.7, 0.75, 0.8, 0.9]},
  'epochs': {'value': 25},
  'device': {'value': 'cuda:2'},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-06, 'max': 6e-05},
  'batch_size': {'distribution': 'q_log_uniform',
   'q': 1,
   'min': 2.772588722239781,
   'max': 3.4657359027997265}}}

In [15]:
sweep_id = wandb.sweep(sweep_config, project="cloud classification")

Create sweep with ID: 6jua7pdw
Sweep URL: https://wandb.ai/leonardtd/cloud%20classification/sweeps/6jua7pdw


---

In [16]:
def train(config=None):

#Init new run
    with wandb.init(config=config):

        config = wandb.config
        train_loader, test_loader, augmentation_loaders = utils.build_dataset_gcd(config.batch_size)

        model = utils.build_model_gatconv(
                                            7, #GCD num classes
                                            config.hidden_dim,
                                            config.num_hidden,
                                            config.num_heads,
                                            config.similarity_threshold,
                                            config.device
                                           )

        optimizer = utils.build_optimizer(config.optimizer, model, config.learning_rate)
        criterion = utils.build_criterion(config.criterion)

        for e in range(config.epochs):
            ### TRAIN DATASET
                preds, targets, loss = engine.train_fn(model, 
                                       train_loader, 
                                       criterion, 
                                       optimizer, 
                                       config.device)
                
                train_acc = accuracy_score(targets, preds)

                ### AUGMENTATION IMAGES
                for _loader in augmentation_loaders:
                    engine.forward_backward_pass(model, 
                                                 _loader, 
                                                 criterion, 
                                                 optimizer, 
                                                 device=config.device)

                test_preds, test_targets, test_loss = engine.eval_fn(model, 
                                           test_loader, 
                                           criterion, 
                                           device=config.device)
                test_acc = accuracy_score(test_targets, test_preds)


                print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format(
                    e+1,
                    train_acc,
                    loss,
                    test_acc,
                    test_loss
                ))

                metrics = {
                            "train/train_loss": loss,
                            "train/train_accuracy": train_acc,
                            "test/test_loss": test_loss,
                            "test/test_accuracy": test_acc,
                          }

                wandb.log(metrics)
                wandb.log({"loss": test_loss, "epoch": e+1})
            
        wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None, 
                                                            preds=test_preds,
                                                            y_true=test_targets,
                                                            class_names=classes)})

---

In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: ysuph7fx with config:
wandb: 	batch_size: 17
wandb: 	criterion: loge
wandb: 	device: cuda:2
wandb: 	epochs: 25
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 5.621303982482521e-06
wandb: 	num_heads: 2
wandb: 	num_hidden: 2
wandb: 	optimizer: nadam
wandb: 	similarity_threshold: 0.9


 28%|██▊       | 167/589 [00:18<00:44,  9.56it/s]